In [1]:
import pandas
nusc_train = 'data/nus_v2/nuscenes_infos_train_mono_front.pkl'
nusc_val = 'data/nus_v2/nuscenes_infos_val_mono_front.pkl'
nusc_val_part = 'data/nus_v2/nuscenes_infos_val_part_mono_front.pkl'
lyft_train = 'data/lyft/lyft_infos_train_mono_front.pkl'
lyft_val = 'data/lyft/lyft_infos_val_mono_front.pkl'

In [2]:
infos = pandas.read_pickle(nusc_train)

In [3]:
import numpy as np
from mmdet3d.structures import LiDARInstance3DBoxes
inst = infos['data_list'][1]['instances'][0]
gt_bbox_3d = np.array(inst['bbox_3d'])
print(gt_bbox_3d)
gt_bbox_3d= LiDARInstance3DBoxes([gt_bbox_3d],box_dim=7,origin=(0.5, 0.5, 0.5))
print(gt_bbox_3d)
print(gt_bbox_3d.gravity_center,gt_bbox_3d.dims,gt_bbox_3d.yaw)
print(gt_bbox_3d.gravity_center.tolist(),gt_bbox_3d.dims.tolist(),gt_bbox_3d.yaw.tolist())

[-7.0452386  18.77374049 -0.10309802  7.516       2.312       3.093
 -0.76545142]
LiDARInstance3DBoxes(
    tensor([[-7.0452, 18.7737, -1.6496,  7.5160,  2.3120,  3.0930, -0.7655]]))
tensor([[-7.0452, 18.7737, -0.1031]]) tensor([[7.5160, 2.3120, 3.0930]]) tensor([-0.7655])
[[-7.045238494873047, 18.773740768432617, -0.10309803485870361]] [[7.515999794006348, 2.312000036239624, 3.0929999351501465]] [-0.7654514312744141]


In [4]:
R = np.array([[0,1,0,0],[-1,0,0,0],[0,0,1,0],[0,0,0,1]])
new2old = np.linalg.inv(R)
indx=0
for info in infos['data_list']:
    # what to change: instances, cam2ego, lidar2cam, ego2global
    insts = info['instances']
    bboxes = []
    for inst in insts:
        bboxes.append(inst['bbox_3d'])
    gt_bbox_3d= LiDARInstance3DBoxes(np.array(bboxes),box_dim=7,origin=(0.5, 0.5, 0.5))
    gt_bbox_3d.rotate((R[:3,:3]).T)
    xyz, lwh, yaw= gt_bbox_3d.gravity_center.tolist(), gt_bbox_3d.dims.tolist(), gt_bbox_3d.yaw.tolist()
    for i in range(len(info['instances'])):
        info['instances'][i]['bbox_3d'] = [*xyz[i],*lwh[i],yaw[i]]

    e2g = info['ego2global']
    c2e = info['images']['CAM_FRONT']['cam2ego']
    l2c = info['images']['CAM_FRONT']['lidar2cam']
    info['images']['CAM_FRONT']['lidar2cam'] = (np.array(l2c) @ new2old).tolist()
    info['images']['CAM_FRONT']['cam2ego'] = (R @ np.array(c2e)).tolist()
    info['ego2global'] = (np.array(e2g) @ new2old).tolist()
    indx+=1
    

In [5]:
import mmengine
mmengine.dump(infos,'data/nus_v2/nuscenes_infos_train_mono_front_bevdet.pkl')

In [ ]:
print(infos['data_list'][0].keys())
print(infos['data_list'][0]['images']['CAM_FRONT'].keys())
print(infos['data_list'][0]['ego2global'])
print(infos['data_list'][0]['images']['CAM_FRONT']['cam2ego'])
print(infos['data_list'][0]['images']['CAM_FRONT']['lidar2cam'])
print(infos['data_list'][0]['instances'][0])
print(infos['data_list'][1]['token'])